In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/Shareddrives/Personal/INR/Playground/Wire Implementation'''

In [23]:
import torch
import torch.nn as nn
from modules import siren, wire, raised_cosine

In [24]:
class Multi_Lyr(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.first_lyr = nn.Linear(in_features=2, out_features=256)
        
        # is_first determines weight initation
        self.siren = siren.SineLayer(in_features=256, 
                                     out_features=256,
                                     omega_0=30,
                                     init_weights=True,
                                     is_first=False)
        self.wire = wire.ComplexGaborLayer(in_features=256, 
                                           out_features=256,
                                           omega0=10,
                                           sigma0=40, 
                                           is_first=False)
        self.raisedCos = raised_cosine.RaisedCosineLayer(in_features=256, 
                                                         out_features=256,
                                                         T0=0.6,
                                                         beta0=0.5,
                                                         is_first=False)
        
        self.last_lyr = nn.Linear(in_features=256, out_features=3)
        
    def forward(self, x):
        # output of wire lyr should be real eg: x = wire(x).real
        x = self.first_lyr(x)
        x = self.siren(x)
        x = self.wire(x.to(torch.cfloat)).real
        x = self.raisedCos(x)
        x = self.last_lyr(x)
        
        return x
        

In [25]:
model = Multi_Lyr().cuda()

In [26]:
input = torch.randn(1024,2).cuda()
out = model(input)

In [27]:
out.shape

torch.Size([1024, 3])